In [ ]:
from azure.ai.ml import load_component
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

In [ ]:
import yaml

tmp_yml_path = "./temp_perf_pipeline_clip_large.yml"
src_yml_path = "./perf_pipeline_clip_large.yml"

input_params_list = [
    {
    "input_dataset_path" : "azureml:10_samples_large:1",
    "concurrent_requests" : 2,
    },
    {
    "input_dataset_path" : "azureml:10_samples_large:1",
    "concurrent_requests" : 4,
    },
    {
    "input_dataset_path" : "azureml:10_samples_large:1",
    "concurrent_requests" : 8,
    },
]


In [ ]:
completed_jobs = []
for input_params in input_params_list:
    with open(src_yml_path, "r") as yaml_file:
        yaml_obj = yaml.safe_load(yaml_file)

    # set dataset
    yaml_obj['jobs']['endpoint']['inputs']['input_dataset']['path'] = input_params['input_dataset_path']
    # set concurrent requests
    yaml_obj['jobs']['endpoint']['inputs']['initial_worker_count'] = input_params['concurrent_requests']
    yaml_obj['jobs']['endpoint']['inputs']['max_worker_count'] = input_params['concurrent_requests']

    with open(tmp_yml_path, "w") as yaml_file:
        yaml_obj = yaml.dump(yaml_obj, yaml_file)

    pipeline_job = load_component(source=tmp_yml_path)
    returned_job = ml_client.jobs.create_or_update(pipeline_job())
    ml_client.jobs.stream(returned_job.name)
    completed_jobs.append(returned_job.name)

In [ ]:
completed_jobs